# Self study 1

In this self study you should work on the code examples below together with the associated questions. The notebook illustrates a basic neural network implementation, where we implement most of the relevant functions from scratch. Except the calculation of gradients, for which we rely on the functionality provided by PyTorch. 

The code illustrates the key concepts involved in the learning neural network. Go carefully through the code before starting to answer the questions at the end.

Part of the code in the notebook is based on the tutorial at https://pytorch.org/tutorials/beginner/nn_tutorial.html

First we import the modules used in this selfstudy

In [2]:
import torch
from pathlib import Path
import requests
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import pickle
import gzip

ModuleNotFoundError: No module named 'requests'

The MNIST database consists of grey scale images of handwritten digits. Each image is of size $28\times 28$; see figure below for an illustration. The data set is divided into a training set, validation set, and test set consisting of $50000$, $10000$, and $10000$ images, respectively; in all data sets the images are labeled with the correct digits. If interested, you can find more information about the MNIST data set at http://yann.lecun.com/exdb/mnist/, including accuracy results for various machine learning methods.

![MNIST DATA](MNIST-dataset.png)

First we download the dataset and unpackage it.

In [ ]:
DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

We then extract the data and store it numpy arrays: x_train, y_train, x_valid, y_valid, x_test, y_test 

In [ ]:
with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), (x_test, y_test)) = pickle.load(f, encoding="latin-1")

Check the shape of the x_valid holding the validation data

In [ ]:
x_valid.shape

The images are stored in rows of length $784$, hence to display the images we need to reshape them to $28\times 28$.

In [ ]:
pyplot.imshow(x_train[0].reshape((28, 28)), cmap="gray")
print(x_train.shape)

In order to take adavntage of PyTorch support for calculating gradients, we need to convert the numpy arrays to PyTorch tensors. See the code example from the last lecture on PyTorch's support for automatic gradient calculation using the back propagation algorithm.

In [ ]:
x_train, y_train, x_valid, y_valid, x_test, y_test = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid, x_test, y_test)
)
n, c = x_train.shape
x_train, x_train.shape, y_train.min(), y_train.max()
print(f"Training data (images): \n {x_train}")
print(f"Training data (labels): \n {y_train}")
print(f"Shape of x_train (now torch tensor) holding the training images: {x_train.shape}")
print(f"Min and max label values: {y_train.min()}, {y_train.max()}")

For the first part of this self study we will specify a neural network, which will encode a [softmax function](https://pytorch.org/tutorials/beginner/nlp/deep_learning_tutorial.html#softmax-and-probabilities). For this we need a (randomly initialized) weight matrix and a bias, and for both of them we need their gradients wrt. our error function (yet to be defined) in order to perform learning. 

We store our weights (and biases) in a matrix structure so that the combination of inputs and weights can be expressed as one single matrix multiplication.

In [ ]:
weights = torch.randn(784, 10) / np.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

Out model specification

In [ ]:
def softmax(x):
    return x.exp() / x.exp().sum(-1).unsqueeze(-1)

# Below @ refers to matrix multiplication
def model(xb):
    return softmax(xb @ weights + bias)

Let's test our model (with our randomly initialized weights) using a so-called batch size of 64 (more on this later in the note book); for the prediction we pick out the first element in the batch.

**Note:** During the lecture we didn't have time to cover learning with batches. Before continuing, please revisit the slides/literature from lecture (in particular Slides 26 and 27) and try to get the gist of this on your own. We will discuss it further at the next lecture on Thursday. 

In [ ]:
batch_size = 64
xb = x_train[0:batch_size]
print(f"Batch shape: {xb.shape}")
preds = model(xb)
print(f"Prediction on first image {preds[0]}")
print(f"Corresponding classification: {preds[0].argmax()}")

Next we define our loss function, in this case the log-loss (or negative log-likelihood):

In [ ]:
def nll(input, target):
    return (-input[range(target.shape[0]), target].log()).mean()

loss_func = nll

# Make a test calculation
yb = y_train[0:batch_size]
print(loss_func(preds,yb))

In the end, we are interested in the accuracy of our model

In [ ]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [ ]:
print(f"Accuracy of model on batch (with random weights): {accuracy(preds, yb)}")

Now we are ready to combine it all and perform learning

In [ ]:
epochs = 10  # how many epochs to train for
lr = 0.01  # learning rate

# We recorded the losses in lists for later plotting
train_losses = []
valid_losses = []

# Iterate for a fixed number of epochs. One epoch is an iteration of the data set, read in chucks of size batch_size
for epoch in range(epochs):
    for batch_idx in range((n - 1) // batch_size + 1):
        
        # pick out the relevant batch
        start_i = batch_idx * batch_size
        end_i = start_i + batch_size
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        
        # Do prediction for all elements in the batch
        pred = model(xb)
        # and calculate the loss
        loss = loss_func(pred, yb)
        
        # Do back propagation to find the gradients
        loss.backward()
        with torch.no_grad():
            # Update the weights
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()
                    
            if batch_idx % 100 == 0:
                with torch.no_grad():
                    train_loss = loss_func(model(x_train), y_train)
                    print(f"Epoch: {epoch}, B-idx: {batch_idx}, Training loss: {train_loss}")
                    train_losses.append(train_loss)

Plot the evolution of the training loss

In [ ]:
plt.plot(range(len(train_losses)), train_losses,'b')

__Exercise:__ 
1. Experiment with different variations of the gradient descent implementation; try varying the learning rate and the batch size. Assuming that you have a fixed time budget (say 2 minutes for learning), what can we then say about the effect of changing the parameters?
2. Implement momentum in the learning algorithm. How does it affect the results? 
3. Try with different initialization schemes for the parameters (e.g. allowing for larger values). How does it affect the behavior of the algorithm?
4. Analyze the behavior of the algorithm on the test set and implement a method for evaluating the accuracy over the entire training/test set (for inspiration, see Line 23 above).